## Capstone Project
### By: Apple Li

#### Step 1: Download packages and dataset of New York boroughs  

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                                                 -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|3_73_pypy|5_73_pypy|4_73_pypy|2_73_pypy|1_73_pypy']
  - geopy -> python[vers

     |████████████████████████████████| 94 kB 6.1 MB/s  eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                          /                                                              failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='5_73_pypy|4_73_pypy|3_73_pypy|2_73_pypy|1_73_pypy|0_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python 

In [3]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

#### Step 2: Extract relevant data from json file and turn into Pandas dataframe that includes borough, neighborhood, latitude, and longitude

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print(neighborhoods['Borough'].unique())
print(neighborhoods.head())
print(neighborhoods['Neighborhood'].count())
print(neighborhoods.shape)



['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']
  Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585
306
(306, 4)


#### Step 3: Conduct exploratory analysis that shows different neighborhoods in New York

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Step 4: Reduce scope of the data to Manhatton only, given the map shows too many neighborhoods across 5 boroughs in New York

In [8]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()
manhattan_data.shape

(40, 4)

In [9]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [10]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### Step 5: Define Foursquare credentials

In [11]:
CLIENT_ID = 'AEDXJUKTKDZZJUAXMMDE2YIPHRSRNMJUPQHPYA0KXBJGL0A0'
CLIENT_SECRET = '2P1Y0TUPYVPI4EI1RNU0SV0KBOZHVU2BSUZAT5VF2U1JMYZR'

VERSION = '20180605'
LIMIT = 10

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: AEDXJUKTKDZZJUAXMMDE2YIPHRSRNMJUPQHPYA0KXBJGL0A0
CLIENT_SECRET:2P1Y0TUPYVPI4EI1RNU0SV0KBOZHVU2BSUZAT5VF2U1JMYZR


#### Step 6: Get venues of each Manhatton neighborhood

In [12]:
CLIENT_ID = 'AEDXJUKTKDZZJUAXMMDE2YIPHRSRNMJUPQHPYA0KXBJGL0A0' # your Foursquare ID
CLIENT_SECRET = '2P1Y0TUPYVPI4EI1RNU0SV0KBOZHVU2BSUZAT5VF2U1JMYZR'

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: AEDXJUKTKDZZJUAXMMDE2YIPHRSRNMJUPQHPYA0KXBJGL0A0
CLIENT_SECRET:2P1Y0TUPYVPI4EI1RNU0SV0KBOZHVU2BSUZAT5VF2U1JMYZR


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
# get venues of all manhattan neighborhood
manhattan_venues=getNearbyVenues(names=manhattan_data['Neighborhood'],latitudes=manhattan_data['Latitude'],longitudes=manhattan_data['Longitude'])
manhattan_venues.head(30)

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.910660,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.910660,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.910660,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.910660,Dunkin',40.877136,-73.906666,Donut Shop
5,Marble Hill,40.876551,-73.910660,Blink Fitness,40.877271,-73.905595,Gym
6,Marble Hill,40.876551,-73.910660,T.J. Maxx,40.877232,-73.905042,Department Store
7,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
8,Marble Hill,40.876551,-73.910660,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
9,Marble Hill,40.876551,-73.910660,Starbucks,40.873755,-73.908613,Coffee Shop


In [15]:
# calculate number of unique venues
len(manhattan_venues['Venue Category'].unique())

330

In [153]:
# filter venues by restaurants (including restaurants, pizza place, and sandwich place)

manhattan_Restaurants = manhattan_venues[manhattan_venues["Venue Category"].str.contains("Restaurant")]    
manhattan_Pizza = manhattan_venues[manhattan_venues["Venue Category"].str.contains("Pizza")]   
manhattan_Sandwich = manhattan_venues[manhattan_venues["Venue Category"].str.contains("Sandwich")]   

manhattan_food = manhattan_Restaurants.append([manhattan_Pizza,manhattan_Sandwich])
manhattan_food.rename(columns = {"Venue Category" : "Restaurants"}, inplace=True)
manhattan_food ['count'] = 1
manhattan_food.head(10)

print ("There are a total of {} restaurants in Manhattan.".format(manhattan_food['count'].sum()))

There are a total of 988 restaurants in Manhattan.


In [143]:
# calculate total number of restaurants per neighborhood
manhattan_food.groupby(manhattan_food['Neighborhood']).count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Restaurants,count
Neighborhood,,,,,,,
Battery Park City,7,7,7,7,7,7,7
Carnegie Hill,22,22,22,22,22,22,22
Central Harlem,16,16,16,16,16,16,16
Chelsea,23,23,23,23,23,23,23
Chinatown,39,39,39,39,39,39,39
Civic Center,24,24,24,24,24,24,24
Clinton,31,31,31,31,31,31,31
East Harlem,18,18,18,18,18,18,18
East Village,43,43,43,43,43,43,43


In [79]:
# Show the number of each type of cuisine (restaurant) in each neighborhood, and calculate total types of cruisines

Table = pd.pivot_table(manhattan_food, values = 'count', index =['Neighborhood'], columns = ['Restaurants'], aggfunc=np.sum)

Table.head(5)

print ("There are {} types cuisines offered across all restaurants  in Manhattan.".format(len(Table. columns)))

There are 76 types cuisines offered across all restaurants  in Manhattan.


In [87]:
# Add 2 columns to the front to show total number of restaurants and total number of cuisines for each neighborhood
Table["Total Restaurants"] = Table.loc[:, "Afghan Restaurant": "Vietnamese Restaurant"].sum(numeric_only = True, axis = 1)
Table["Type of Cuisines"] = Table.loc[:, "Afghan Restaurant": "Vietnamese Restaurant"].count(numeric_only = True, axis = 1)


Table_new = Table[['Total Restaurants', 'Type of Cuisines'] + [c for c in Table if c not in ['Total Restaurants', 'Type of Cuisines']]]
Table_new.head()


Restaurants,Total Restaurants,Type of Cuisines,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Battery Park City,7.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carnegie Hill,22.0,17,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
Central Harlem,16.0,10,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Chelsea,23.0,15,NaN,NaN,4.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Chinatown,39.0,21,NaN,NaN,3.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,1.0,NaN,7.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0


In [119]:
# Sort neighborhoods by total number restaurants
Table_total = Table_new.sort_values(by = "Total Restaurants", ascending = False)
Table_total.head()

Restaurants,Total Restaurants,Type of Cuisines,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Greenwich Village,45.0,23,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,2.0
East Village,43.0,25,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,2.0,NaN,2.0
West Village,39.0,16,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,11.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chinatown,39.0,21,NaN,NaN,3.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,1.0,NaN,7.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0
Noho,38.0,21,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0


In [120]:
# Sort neighborhoods by variety of cuisines
Table_total = Table_new.sort_values(by = "Type of Cuisines", ascending = False)
Table_total.head()

Restaurants,Total Restaurants,Type of Cuisines,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
East Village,43.0,25,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,2.0,NaN,2.0
Greenwich Village,45.0,23,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,2.0
Murray Hill,38.0,22,NaN,NaN,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,1.0,1.0,NaN,6.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,NaN,2.0
Chinatown,39.0,21,NaN,NaN,3.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,1.0,NaN,7.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0
Upper West Side,34.0,21,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,3.0,1.0,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,1.0,NaN,1.0


In [122]:
# Add the total number of restaurants by cuisine type to data table, and the maximum number of restaurants of one type in a neighborhood

Table_total.loc['Manhattan Total'] = Table_total["East Village":"Roosevelt Island"].sum(numeric_only = True, axis = 0)
Table_total.loc ['Maximum Number of Restaurants in a Neighborhood'] = Table_total["East Village":"Roosevelt Island"].max(numeric_only = True, axis = 0)

Table_total.tail()


Restaurants,Total Restaurants,Type of Cuisines,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Battery Park City,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Marble Hill,5.0,4.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Roosevelt Island,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Manhattan Total,988.0,565.0,1.0,3.0,73.0,1.0,5.0,16.0,4.0,2.0,2.0,1.0,1.0,10.0,46.0,9.0,1.0,4.0,4.0,2.0,1.0,1.0,3.0,9.0,4.0,3.0,44.0,2.0,17.0,5.0,1.0,5.0,27.0,2.0,121.0,3.0,37.0,3.0,2.0,25.0,1.0,16.0,1.0,3.0,28.0,54.0,5.0,2.0,1.0,1.0,25.0,1.0,2.0,2.0,4.0,71.0,12.0,21.0,1.0,47.0,1.0,36.0,3.0,1.0,1.0,3.0,11.0,44.0,1.0,3.0,2.0,9.0,29.0,7.0,2.0,16.0,1.0,20.0
Maximum Number of Restaurants in a Neighborhood,45.0,25.0,1.0,3.0,5.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,7.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,3.0,1.0,1.0,4.0,3.0,1.0,11.0,1.0,4.0,2.0,1.0,15.0,1.0,2.0,1.0,2.0,4.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,6.0,2.0,3.0,1.0,6.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,2.0,1.0,2.0


In [141]:
# create a formula that returns the neighborhood that has the greatest number of a select restaurant type

def Find_Restaurant (R):
    Neighborhood = Table_new[R].idxmax()
    return print("You can find the most {} in {}.".format(R, Neighborhood))

R1 = "Korean Restaurant"
Find_Restaurant (R1)

R2 = "Italian Restaurant"
Find_Restaurant (R2)

R3 = "American Restaurant"
Find_Restaurant (R3)

R4 = "French Restaurant"
Find_Restaurant (R4)

R5 = "Mexican Restaurant"
Find_Restaurant (R5)

You can find the most Korean Restaurant in Midtown South.
You can find the most Italian Restaurant in Greenwich Village.
You can find the most American Restaurant in West Village.
You can find the most French Restaurant in Carnegie Hill.
You can find the most Mexican Restaurant in East Harlem.


In [178]:
# create map of East Village's restaurants using latitude and longitude values

address_EV = 'East Village, NY'

geolocator_EV = Nominatim(user_agent="ny_explorer")
location_EV = geolocator.geocode(address_EV)
latitude_EV = location_EV.latitude
longitude_EV = location_EV.longitude
map_East_Village = folium.Map(location=[latitude_EV, longitude_EV], zoom_start=16)

map_East_Village


In [167]:
manhattan_food.head()

East_Village = manhattan_food [manhattan_food['Neighborhood'] == "East Village"]
East_Village.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Restaurants,count
1450,East Village,40.727847,-73.982226,Smør,40.729295,-73.981521,Scandinavian Restaurant,1
1451,East Village,40.727847,-73.982226,Kura,40.726803,-73.983444,Japanese Restaurant,1
1453,East Village,40.727847,-73.982226,Cafe Mogador,40.727277,-73.984505,Moroccan Restaurant,1
1458,East Village,40.727847,-73.982226,Westville East,40.728428,-73.981894,American Restaurant,1
1460,East Village,40.727847,-73.982226,Thursday Kitchen,40.727661,-73.983761,Korean Restaurant,1


In [186]:
# add markers to map
for lat, lng, label in zip(East_Village['Venue Latitude'], East_Village['Venue Longitude'], East_Village['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_East_Village)  
    
map_East_Village

In [179]:
# create map of East Village's restaurants using latitude and longitude values

address_GV = 'Greenwich Village, NY'

geolocator_GV = Nominatim(user_agent="ny_explorer")
location_GV = geolocator.geocode(address_GV)
latitude_GV = location_GV.latitude
longitude_GV = location_GV.longitude
map_Greenwich_Village = folium.Map(location=[latitude_GV, longitude_GV], zoom_start=16)

map_Greenwich_Village

In [180]:
Greenwich_Village = manhattan_food [manhattan_food['Neighborhood'] == "Greenwich Village"]
Greenwich_Village.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Restaurants,count
1350,Greenwich Village,40.726933,-73.999914,Carbone,40.727903,-74.000136,Italian Restaurant,1
1358,Greenwich Village,40.726933,-73.999914,Raoul's Restaurant,40.726224,-74.002093,French Restaurant,1
1360,Greenwich Village,40.726933,-73.999914,DomoDomo,40.728007,-74.001683,Sushi Restaurant,1
1364,Greenwich Village,40.726933,-73.999914,Coco Pazzo Kitchen & Restaurant,40.726036,-74.001507,Italian Restaurant,1
1366,Greenwich Village,40.726933,-73.999914,Cocotte,40.725811,-74.001886,French Restaurant,1


In [184]:
# add markers to map
for lat, lng, label in zip(Greenwich_Village['Venue Latitude'], Greenwich_Village['Venue Longitude'], Greenwich_Village['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Greenwich_Village)  
    
map_Greenwich_Village